# Лабораторная работа 3

## ФИО: Фам Данг Чунг Нгиа

##Группа: P3321

## ИСу 374806

## Configuration

### Installing libraries

In [1]:
%pip install --upgrade --no-cache-dir pip setuptools wheel

# core libs
%pip install --force-reinstall --no-cache-dir \
  "numpy==1.26.4" "pandas==2.2.2" "scipy==1.13.1" \
  "mne==1.10.2" "pyriemann==0.7" "moabb==1.2.0" \
  "scikit-learn==1.5.2" "matplotlib==3.9.2"

# PyTorch GPU
%pip install --no-cache-dir --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

%pip install --no-cache-dir tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 189.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 367.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 324.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 348.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 243.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 180.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 219.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 342.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 184.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 281.0 MB/s  0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting unin

Looking in indexes: https://download.pytorch.org/whl/cu121


### Import libs

In [ ]:
# Core
import os, gc
import numpy as np
import pandas as pd

# Config for my drive & MNE/MOABB
from google.colab import drive
import mne
from moabb.datasets import BI2013a
from moabb.paradigms import P300

# Signal processing
from scipy.signal import butter, sosfiltfilt, decimate

# PyTorch (no nn/optim)
import torch
from torch import Tensor

# Split & metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score, balanced_accuracy_score,
    roc_auc_score, confusion_matrix,
)

### Config to save dataset on my drive


In [1]:
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/mne_data"
os.makedirs(DATA_DIR, exist_ok=True)

os.environ["MNE_DATA"] = DATA_DIR
os.environ["MNE_DATASETS_PATH"] = DATA_DIR
os.environ["MOABB_DATA_PATH"] = DATA_DIR
os.environ["MOABB_DOWNLOAD_DIR"] = DATA_DIR
os.environ["MOABB_CACHE_DIR"] = DATA_DIR

mne.set_config("MNE_DATA", DATA_DIR, set_env=True)
mne.set_config("MNE_DATASETS_PATH", DATA_DIR, set_env=True)

print("ENV MNE_DATA:", os.getenv("MNE_DATA"))
print("mne.get_config('MNE_DATA'):", mne.get_config("MNE_DATA"))

Mounted at /content/drive
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Could not read the /root/.mne/mne-python.json json file during the writing. Assuming it is empty. Got: Expecting value: line 1 column 1 (char 0)
ENV MNE_DATA: /content/drive/MyDrive/mne_data
mne.get_config('MNE_DATA'): /content/drive/MyDrive/mne_data


/tmp/ipython-input-3433513575.py:15: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_PATH"
  mne.set_config("MNE_DATASETS_PATH", DATA_DIR, set_env=True)


In [2]:
# Symlink:  /root/mne_data -> Drive
import os, subprocess, shutil, pathlib

if os.path.islink("/root/mne_data") or os.path.exists("/root/mne_data"):
    shutil.rmtree("/root/mne_data", ignore_errors=True)

#  symlink
pathlib.Path("/root").mkdir(parents=True, exist_ok=True)
subprocess.run(["ln", "-s", DATA_DIR, "/root/mne_data"], check=True)

print("Symlink OK:", os.path.islink("/root/mne_data"), "->", os.readlink("/root/mne_data"))

Symlink OK: True -> /content/drive/MyDrive/mne_data


## Install the dataset

In [3]:
SUBJECTS = [1, 2, 3, 4]
ds = BI2013a(NonAdaptive=True, Adaptive=True, Training=True, Online=True)
paradigm = P300(resample=None)

# get data from drive, download only data that is not yet available
X, y_str, meta = paradigm.get_data(dataset=ds, subjects=SUBJECTS)

print("X shape:", X.shape)           # (N_trials, N_channels, N_times)
print("y sample:", y_str[:10])
print(meta.head())
print("Label counts:", pd.Series(y_str).value_counts().to_dict())

/usr/local/lib/python3.12/dist-packages/moabb/datasets/download.py:56: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BRAININVADERS2013_PATH"
  set_config(key, get_config("MNE_DATA"))
/usr/local/lib/python3.12/dist-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 480 events (all good), 0 – 1 s (baseline off), ~30.1 MiB, data loaded,
 'Target': 80
 'NonTarget': 400>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.12/dist-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 192 events (all good), 0 – 1 s (baseline off), ~12.0 MiB, data loaded,
 'Target': 32
 'NonTarget': 160>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.12/dist-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 480 events (all good), 0 – 1 s (baseline off), ~30.1 MiB, data loaded,
 'Target': 80
 'NonTarget': 400>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.12/dist-packages/moabb/datasets/prepro

X shape: (45312, 16, 513)
y sample: ['Target' 'NonTarget' 'NonTarget' 'NonTarget' 'NonTarget' 'NonTarget'
 'NonTarget' 'Target' 'NonTarget' 'NonTarget']
   subject session run
0        1       0   0
1        1       0   0
2        1       0   0
3        1       0   0
4        1       0   0
Label counts: {'NonTarget': 37760, 'Target': 7552}


X (N_trials, N_channels, N_times) =  (45312, 16, 513)

* N_trials = 45 312: number of trials/epoch (train ~ epoch)

* N_channels = 16: the number of EEG electrodes used to conduct measurement

* N_times = 513: sample number of times in an epoch (total time = fs * N_times)

y ('NonTarget', 'Target') = (37760, 7552)

* each trial has a label y

* Target -> + has P300

* Non-Target -> none P300

## Dataset preprocessing

Guard: make sure there are  X, y_str, meta

In [4]:
assert 'X' in globals() and 'y_str' in globals() and 'meta' in globals(), \
    "Need to run the previous block (create X, y_str, meta)."

Preparation of parameters

In [5]:
# ==== fs, tmin (iloc[0] - first sample time of epoch) ====
fs = float(meta["sfreq"].iloc[0]) if "sfreq" in meta.columns else 256.0
tmin = float(meta["tmin"].iloc[0]) if "tmin" in meta.columns else -0.2

# ==== parameters pipeline ====
lo, hi, order = 0.1, 15.0, 4
target_fs = 128                     # decimate
win = (0.30, 0.60)                  # Windowing EEG
base = (-0.2, 0.0)                  # baseline
n_bins = 6                          # bin feature
BATCH = 64                          # batch per trial

# ==== Prepare time Index once ====
N, C, T = X.shape
times = tmin + np.arange(T) / fs
base_mask = (times >= base[0]) & (times <= base[1])
win_mask  = (times >= win[0])  & (times <= win[1])

# ==== filters ====
nyq = fs * 0.5
sos = butter(order, [lo/nyq, hi/nyq], btype='band', output='sos')

Mean Bin Features

In [6]:
def feats_from_window(Xw, n_bins=6):
    # Xw: (b, C, Tw) float32
    bsz, Cc, Tw = Xw.shape
    edges = np.linspace(0, Tw, n_bins+1, dtype=int)
    F = np.empty((bsz, Cc, n_bins, 4), dtype=np.float32)
    for i in range(n_bins):
        s, e = edges[i], edges[i+1]
        if e - s <= 0:
            F[:, :, i, :] = 0.0
            continue
        seg = Xw[:, :, s:e]                           # (b,C,L)
        m  = seg.mean(axis=2)
        mx = seg.max(axis=2)
        mn = seg.min(axis=2)
        half = (e - s) // 2
        if half == 0:
            sl = np.zeros_like(m, dtype=np.float32)
        else:
            sl = seg[:, :, :half].mean(axis=2) - seg[:, :, half:].mean(axis=2)
        F[:, :, i, 0] = m
        F[:, :, i, 1] = mx
        F[:, :, i, 2] = mn
        F[:, :, i, 3] = sl
    return F.reshape(bsz, Cc * n_bins * 4)

Window-length estimation, feature preallocation, and label encoding

In [7]:
Tw_est = int(win_mask.sum()) if not isinstance(win_mask, slice) else X.shape[2]
X_feat = np.empty((N, C * n_bins * 4), dtype=np.float32)
y = np.where(np.array(y_str) == "Target", 1.0, -1.0).astype(np.float32)

Dataset preprocessing according to batch

In [8]:
fs_eff = fs   # keep the fs fixed for the pipeline
for s in range(0, N, BATCH):
    e = min(N, s + BATCH)
    # get batch and press float32 (reduce 1/2 RAM)
    xb = np.asarray(X[s:e], dtype=np.float32, order='C')   # (b,C,T)

    # 1) Band-pass (sosfiltfilt per trial)
    for i in range(xb.shape[0]):
        xb[i] = sosfiltfilt(sos, xb[i], axis=1)

    # 2) Baseline correction
    if isinstance(base_mask, np.ndarray) and base_mask.any():
        base_mean = xb[:, :, base_mask].mean(axis=2, keepdims=True)
        xb -= base_mean

    # 3) Window eeg
    if isinstance(win_mask, np.ndarray):
        xw = xb[:, :, win_mask].copy()
    else:
        xw = xb

    # 4) Decimate to target_fs
    fs_dec = fs_eff
    if int(fs_eff) != target_fs:
        ratio = int(round(fs_eff / target_fs))
        if ratio >= 2 and abs(fs_eff/ratio - target_fs) < 1e-6:
            xwd = []
            for i in range(xw.shape[0]):
                xi = decimate(xw[i], ratio, axis=1, ftype='fir', zero_phase=True).astype(np.float32, copy=False)
                xwd.append(xi)
            xw = np.stack(xwd, axis=0)   # (batch, C, Tw//ratio)
            del xwd
            fs_dec = fs_eff / ratio
            if s == 0:
                print(f"Decimated to fs={fs_dec} Hz (ratio={ratio}) | shape batch:", xw.shape)
        else:
            if s == 0:
                print("Ignore decimate because fs does not divide logically.")

    # 5) Extract feature per bin
    X_feat[s:e] = feats_from_window(xw, n_bins=n_bins)

    # clean
    del xb, xw
    gc.collect()

print("Features:", X_feat.shape, " | Pos%:", float((y==1).mean()))


Decimated to fs=128.0 Hz (ratio=2) | shape batch: (64, 16, 39)
Features: (45312, 384)  | Pos%: 0.16666666666666666


## Model SVM

In [19]:
class LinearSVMTorch:
    def __init__(self, lr=1e-3, C=1.0, reg=1e-4, batch_size=128,
                 n_epochs=10, shuffle=True, random_state=123, device=None):
        # hyper-params
        self.lr = lr
        self.C = C              # coefficient for hinge loss
        self.reg = reg          # L2 weight (same role as in NumPy version)
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.shuffle = shuffle
        self.random_state = random_state
        # runtime
        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        # parameters (initialized in fit)
        self.W = None  # (D, 1) tensor, requires_grad=True
        self.b = None  # () tensor, requires_grad=True

    # --- helpers ---
    @staticmethod
    def _to_tensor(x, device):
        """Convert numpy/torch to torch.float32 on device."""
        if isinstance(x, Tensor):
            return x.to(device=device, dtype=torch.float32, non_blocking=True)
        return torch.as_tensor(x, dtype=torch.float32, device=device)

    def _batch_iter(self, X: torch.Tensor, y: torch.Tensor):
      """Yield mini-batches with optional shuffling (pure torch)."""
      N = X.shape[0]
      if self.shuffle:
          # make generator on CPU, sample on CPU, then move to device
          g = torch.Generator(device="cpu")
          g.manual_seed(self.random_state)
          idx = torch.randperm(N, generator=g)           # CPU tensor
          idx = idx.to(self.device, non_blocking=True)   # move to GPU/CPU target
      else:
          idx = torch.arange(N, device=self.device)
      for s in range(0, N, self.batch_size):
          j = idx[s:s + self.batch_size]
          yield X[j], y[j]


    # --- api ---
    def fit(self, X, y, X_val=None, y_val=None, verbose=True):
        """
        X: (N, D) float32/np, y: (N,) in {-1,+1}
        """
        torch.manual_seed(self.random_state)

        X = self._to_tensor(X, self.device)  # (N, D)
        y = self._to_tensor(y, self.device)  # (N,)

        N, D = X.shape
        # init params (zeros like NumPy impl)
        self.W = torch.zeros(D, 1, device=self.device, dtype=torch.float32, requires_grad=True)
        self.b = torch.zeros(1, device=self.device, dtype=torch.float32, requires_grad=True)

        # (optional) validation tensors
        Xv = self._to_tensor(X_val, self.device) if X_val is not None else None
        yv = self._to_tensor(y_val, self.device) if y_val is not None else None

        for epoch in range(1, self.n_epochs + 1):
            total_loss = 0.0
            seen = 0

            for Xb, yb in self._batch_iter(X, y):
                # ----- forward -----
                # scores: (B,)  = Xb @ W + b
                scores = (Xb @ self.W).squeeze(-1) + self.b  # (B,)
                # margins = 1 - y * score
                margins = 1.0 - yb * scores                  # (B,)
                # hinge part (mean over violations)
                hinge = torch.clamp(margins, min=0.0)        # (B,)
                hinge_loss = hinge.mean()
                # L2 on weights (same as 0.5 * reg * ||w||^2)
                l2 = 0.5 * self.reg * (self.W.square().sum())
                # total loss
                loss = self.C * hinge_loss + l2

                # ----- backward -----
                # zero grads
                if self.W.grad is not None: self.W.grad.zero_()
                if self.b.grad is not None: self.b.grad.zero_()
                loss.backward()

                # ----- manual SGD update -----
                with torch.no_grad():
                    self.W -= self.lr * self.W.grad
                    self.b -= self.lr * self.b.grad

                total_loss += float(loss.item()) * Xb.shape[0]
                seen += Xb.shape[0]

            avg_loss = total_loss / max(seen, 1)

            if verbose:
                if Xv is not None and yv is not None:
                    val_pred = self.predict(Xv)  # returns torch tensor {-1,+1}
                    val_acc = (val_pred == yv).float().mean().item()
                    print(f"Epoch {epoch:02d}: loss={avg_loss:.5f} | val_acc={val_acc:.4f}")
                else:
                    print(f"Epoch {epoch:02d}: loss={avg_loss:.5f}")

        return self

    @torch.no_grad() # turn off autograd in this function -> dont use RAM
    def decision_function(self, X):
        """Return raw scores (no threshold). Accepts numpy or tensor. Output: torch tensor (N,)."""
        X = self._to_tensor(X, self.device)
        scores = (X @ self.W).squeeze(-1) + self.b
        return scores

    @torch.no_grad() # turn off autograd in this function -> dont use RAM
    def predict(self, X, thr: float = 0.0):
        """Return labels in {-1,+1} on the SAME device as model/input."""
        s = self.decision_function(X)  # stays on self.device
        return torch.where(
            s >= thr,
            torch.tensor(1.0, device=s.device),
            torch.tensor(-1.0, device=s.device),
        )

## Train and test

Dataset split and standardization

In [15]:
X_tr, X_tmp, y_tr, y_tmp = train_test_split(
    X_feat, y, test_size=0.30, random_state=42, stratify=y
)
X_va, X_te, y_va, y_te = train_test_split(
    X_tmp, y_tmp, test_size=0.50, random_state=42, stratify=y_tmp
)

print("Train/Val/Test:", X_tr.shape, X_va.shape, X_te.shape)

Train/Val/Test: (31718, 384) (6797, 384) (6797, 384)


Custom standardization function

In [16]:
class StandardScalerLite:
    def fit(self, X):
        self.mean_ = X.mean(axis=0, keepdims=True)
        self.std_  = X.std(axis=0, keepdims=True) + 1e-8
        return self
    def transform(self, X):
        return (X - self.mean_) / self.std_
    def fit_transform(self, X):
        return self.fit(X).transform(X)

scaler = StandardScalerLite()
X_tr = scaler.fit_transform(X_tr)   # fit on train
X_va = scaler.transform(X_va)       # apply to val
X_te = scaler.transform(X_te)       # apply to test

Training & selecting threshold, metrics

In [20]:
svm = LinearSVMTorch(lr=2e-3, C=1.0, reg=1e-4, batch_size=256, n_epochs=12,
                     shuffle=True, random_state=42)
svm.fit(X_tr, y_tr, X_val=X_va, y_val=y_va, verbose=True)

Epoch 01: loss=0.92860 | val_acc=0.7389
Epoch 02: loss=0.79448 | val_acc=0.7662
Epoch 03: loss=0.68679 | val_acc=0.8095
Epoch 04: loss=0.59252 | val_acc=0.8339
Epoch 05: loss=0.50529 | val_acc=0.8345
Epoch 06: loss=0.42765 | val_acc=0.8333
Epoch 07: loss=0.37007 | val_acc=0.8333
Epoch 08: loss=0.34517 | val_acc=0.8333
Epoch 09: loss=0.33876 | val_acc=0.8333
Epoch 10: loss=0.33692 | val_acc=0.8333
Epoch 11: loss=0.33647 | val_acc=0.8333
Epoch 12: loss=0.33606 | val_acc=0.8333


In [21]:
# decision_function returns torch tensor; convert to numpy for sklearn
scores_va = svm.decision_function(X_va).cpu().numpy()

ths = np.quantile(scores_va, np.linspace(0.05, 0.95, 19))
best_th, best_f1 = 0.0, -1.0
for th in ths:
    y_hat = np.where(scores_va >= th, 1.0, -1.0)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_va, y_hat, average='binary', zero_division=0, pos_label=1.0
    )
    if f1 > best_f1:
        best_f1, best_th = f1, th

print(f"Best threshold on VAL: {best_th:.6f} (F1={best_f1:.4f})")

Best threshold on VAL: -1.005146 (F1=0.4477)


In [25]:
scores_te = svm.decision_function(X_te).cpu().numpy()
y_pred = np.where(scores_te >= best_th, 1.0, -1.0)

# For some metrics we use {0,1}
y_te01 = (y_te == 1.0).astype(int)
y_pred01 = (y_pred == 1.0).astype(int)

acc  = accuracy_score(y_te, y_pred)
bacc = balanced_accuracy_score(y_te01, y_pred01)
prec, rec, f1, _ = precision_recall_fscore_support(y_te, y_pred, average='binary',
                                                   zero_division=0, pos_label=1.0)
try:
    auc = roc_auc_score(y_te01, scores_te)
except Exception:
    auc = float('nan')

cm = confusion_matrix(y_te, y_pred, labels=[-1.0, 1.0])

print("==== TEST METRICS ====")
print(f"Threshold used   : {best_th:.6f}")
print(f"Accuracy         : {acc:.4f}")
print(f"Balanced Acc     : {bacc:.4f}")
print(f"Precision/Recall/F1: {prec:.4f}/ {rec:.4f}/ {f1:.4f}")
print(f"ROC-AUC          : {auc:.4f}")

==== TEST METRICS ====
Threshold used   : -1.005146
Accuracy         : 0.7449
Balanced Acc     : 0.6937
Precision/Recall/F1: 0.3497/ 0.6169/ 0.4464
ROC-AUC          : 0.7645
